<a href="https://colab.research.google.com/github/hyojunyee/kita_2404/blob/main/m3_%EB%B6%84%EC%84%9D%20%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC/Task/pandas_%EC%8B%A4%EC%8A%B5_titanic_0603.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[실습] titanic3.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 데이터 불러오기
file_path = '/content/drive/MyDrive/kdt_240424/m3_분석라이브러리/m_3_datas/titanic3.csv'
# file_path = 'titanic3.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB
None


In [ ]:
print(df.head())

   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

     age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
0  29.00      0      0   24160  211.3375       B5        S    2    NaN   
1   0.92      1      2  113781  151.5500  C22 C26        S   11    NaN   
2   2.00      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
3  30.00      1      2  113781  151.5500  C22 C26        S  NaN  135.0   
4  25.00      1      2  113781  151.5500  C22 C26        S  NaN    NaN   

                         home.dest  
0                     St Louis, MO  


In [ ]:
# 결측치 확인
print(df.isnull().sum())

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


In [ ]:
# age 결측치 측정할때 전체 평균을 구해주는 것 보다 호칭에 평균을 대입해주는 것 생각해보기

In [ ]:
# 필요 없는 열 제거
df.drop(columns = ['name','ticket','body','home.dest','boat','pclass','cabin'], inplace=True)
df.head()

,survived,sex,age,sibsp,parch,fare,embarked
0,1,female,29.00,0,0,211.3375,S
1,1,male,0.92,1,2,151.5500,S
2,0,female,2.00,1,2,151.5500,S
3,0,male,30.00,1,2,151.5500,S
4,0,female,25.00,1,2,151.5500,S


In [ ]:
# 결측치 처리
df['age'].fillna(df['age'].mean(), inplace=True)
df['fare'].fillna(df['fare'].mean(), inplace=True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)
df.head()

,survived,sex,age,sibsp,parch,fare,embarked
0,1,female,29.00,0,0,211.3375,S
1,1,male,0.92,1,2,151.5500,S
2,0,female,2.00,1,2,151.5500,S
3,0,male,30.00,1,2,151.5500,S
4,0,female,25.00,1,2,151.5500,S


In [ ]:
# age_group을 6개 범주로 확대
bins = [0,5,12,18,35,60,100]
labels = ['Infant','child','Teenager','Young Adult','Adult','Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)
df.drop(columns=['age'], inplace=True)
df.head()

,survived,sex,sibsp,parch,fare,embarked,age_group
0,1,female,0,0,211.3375,S,Young Adult
1,1,male,1,2,151.5500,S,Infant
2,0,female,1,2,151.5500,S,Infant
3,0,male,1,2,151.5500,S,Young Adult
4,0,female,1,2,151.5500,S,Young Adult


In [ ]:
# fare를 3개 범주로 변환
fare_bins = [0, 30, 100, 600]
fare_labels = ['Low', 'Medium', 'High']
df['fare_group'] = pd.cut(df['fare'], bins = fare_bins, labels=fare_labels)
df.drop(columns=['fare'], inplace=True)
df.head()

,survived,sex,sibsp,parch,embarked,age_group,fare_group
0,1,female,0,0,S,Young Adult,High
1,1,male,1,2,S,Infant,High
2,0,female,1,2,S,Infant,High
3,0,male,1,2,S,Young Adult,High
4,0,female,1,2,S,Young Adult,High


In [ ]:
# sibsp와 parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['sibsp'] + df['parch'] + 1   # 자기 자신 포함
df.drop(columns=['sibsp', 'parch'], inplace=True)

In [ ]:
# One-Hot Encoding
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['age_group', 'fare_group', 'sex', 'embarked']

# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫번쨰 카테고리는 drop_first=True로 제거
    df = pd.get_dummies(df, columns=[column], drop_first=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   survived               1309 non-null   int64
 1   family_size            1309 non-null   int64
 2   age_group_child        1309 non-null   bool 
 3   age_group_Teenager     1309 non-null   bool 
 4   age_group_Young Adult  1309 non-null   bool 
 5   age_group_Adult        1309 non-null   bool 
 6   age_group_Senior       1309 non-null   bool 
 7   fare_group_Medium      1309 non-null   bool 
 8   fare_group_High        1309 non-null   bool 
 9   sex_male               1309 non-null   bool 
 10  embarked_Q             1309 non-null   bool 
 11  embarked_S             1309 non-null   bool 
dtypes: bool(10), int64(2)
memory usage: 33.4 KB


In [ ]:
# 타겟 변수와 피처 변수 설정
X = df.drop('survived', axis=1)
y = df['survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

(1047, 11) (262, 11) (1047,) (262,)
Accuracy: 0.767175572519084
